# Проект: классификация

In [1184]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from  sklearn.ensemble import IsolationForest
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing  import LabelEncoder
from sklearn import linear_model 
from sklearn import tree 
from sklearn import ensemble 
from sklearn import metrics 
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split 
from sklearn.feature_selection import SelectKBest, f_classif

## Часть 1. Знакомство с данными, обработка пропусков и выбросов

Сначала вставим для удобства сюда описание признаков:
* Данные о клиентах банка:
    * age (возраст);
    * job (сфера занятости);
    * marital (семейное положение);
    * education (уровень образования);
    * default (имеется ли просроченный кредит);
    * housing (имеется ли кредит на жильё);
    * loan (имеется ли кредит на личные нужды);
    * balance (баланс).
* Данные, связанные с последним контактом в контексте текущей маркетинговой кампании:
    * contact (тип контакта с клиентом);
    * month (месяц, в котором был последний контакт);
    * day (день, в который был последний контакт);
    * duration (продолжительность контакта в секундах).
* Прочие признаки:
    * campaign (количество контактов с этим клиентом в течение текущей кампании);
    * pdays (количество пропущенных дней с момента последней маркетинговой кампании до контакта в текущей кампании);
    * previous (количество контактов до текущей кампании)
    * poutcome (результат прошлой маркетинговой кампании).
* deposit, которая определяет, согласится ли клиент открыть депозит в банке. Именно её мы будем пытаться предсказать в данном кейсе.

### Задание 1

In [1185]:
df = pd.read_csv('bank_fin.csv', sep = ';')

Посмотрим сначала на признаки и их типы

In [1186]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11162 non-null  int64 
 1   job        11162 non-null  object
 2   marital    11162 non-null  object
 3   education  11162 non-null  object
 4   default    11162 non-null  object
 5   balance    11137 non-null  object
 6   housing    11162 non-null  object
 7   loan       11162 non-null  object
 8   contact    11162 non-null  object
 9   day        11162 non-null  int64 
 10  month      11162 non-null  object
 11  duration   11162 non-null  int64 
 12  campaign   11162 non-null  int64 
 13  pdays      11162 non-null  int64 
 14  previous   11162 non-null  int64 
 15  poutcome   11162 non-null  object
 16  deposit    11162 non-null  object
dtypes: int64(6), object(11)
memory usage: 1.4+ MB


Посмотрим на на верх таблицы, чтобы сориетироваться, что там за данные

In [1187]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,"2 343,00 $",yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,"45,00 $",no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,"1 270,00 $",yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,"2 476,00 $",yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,"184,00 $",no,no,unknown,5,may,673,2,-1,0,unknown,yes


Навскидку видно, что есть много категориальных признаков, и из надо будет перекодировать. Balance по сути числовой, надо перевести в число.Посмотрим describe

In [1188]:
df.describe(include='object')

,job,marital,education,default,balance,housing,loan,contact,month,poutcome,deposit
count,11162,11162,11162,11162,11137,11162,11162,11162,11162,11162,11162
unique,12,3,4,2,3802,2,2,3,12,4,2
top,management,married,secondary,no,0 $,no,no,cellular,may,unknown,no
freq,2566,6351,5476,10994,774,5881,9702,8042,2824,8326,5873


In [1189]:
# исследуйте данные на предмет пропусков. Где есть пропущенные значения? Сколько их?
df.isnull().sum().sort_values(ascending=False)

balance      25
age           0
day           0
poutcome      0
previous      0
pdays         0
campaign      0
duration      0
month         0
contact       0
job           0
loan          0
housing       0
default       0
education     0
marital       0
deposit       0
dtype: int64

Итого имеем 25 пропусков в признаке balance и все, других пропусков нет

### Задание 2

In [1190]:
# есть ли в признаке job пропущенные значения? Возможно, они обозначены каким-то специальным словом?
jod_df = df.groupby('job')['job'].value_counts().sort_values(ascending=False)
jod_df

job            job          
management     management       2566
blue-collar    blue-collar      1944
technician     technician       1823
admin.         admin.           1334
services       services          923
retired        retired           778
self-employed  self-employed     405
student        student           360
unemployed     unemployed        357
entrepreneur   entrepreneur      328
housemaid      housemaid         274
unknown        unknown            70
Name: job, dtype: int64

Видим, что пропуски кроются за словом unknown

### Задание 3

In [1191]:
# преобразуйте признак balance таким образом, чтобы он корректно считывался, как вещественное число (float)
df['balance'] = df['balance'].apply(lambda x: str(x).replace(' ',''))
df['balance'] = df['balance'].apply(lambda x: str(x).replace('$',''))
df['balance'] = df['balance'].apply(lambda x: str(x).replace(',','.')).astype('float')

In [1192]:
df.balance.mean()

1529.1290293615875

### Задание 4

In [1193]:
# обработайте пропуски в признаки balance , заменив их на медианные значения по данному признаку
df['balance'] = df['balance'].fillna(df['balance'].median())
df.balance.mean()

1526.9360329690019

### Задание 5

In [1194]:
# обработайте пропуски в категориальных признаках: job и education, заменив их на модальные значения
j_mode = df['job'].mode()[0]
e_mode = df['education'].mode()[0]
df['job'] = df['job'].apply(lambda x: j_mode if x == 'unknown' else x)
df['education'] = df['education'].apply(lambda x: e_mode if x == 'unknown' else x)


In [1195]:
#Выполним проверку к этому заданию. Расчитаем средний баланс по самой популярной професии + работе
df[ df['job'] == j_mode][df['education'] == e_mode ]['balance'].mean().round(3)

1598.883

### Задание 6

In [1196]:
# удалите все выбросы для признака balance
def outliers_iqr_mod(data, feature, log_scale=False, left=1.5, right=1.5): #функция которая по сути возвращает 2 таблицы: выбросы и без выбросов по признаку 
    if log_scale:  #если указываем, что хотим использовать логарифмическую шкалу, что логарифмируем данные по параметру
        x = np.log(data[feature]) #единицу добавляем, чтобы не наткнуться на нулевые расстояния логарифма которых нет
    else:
        x = data[feature] 
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75), # считаем квантили
    iqr = quartile_3 - quartile_1 # считаем межквартильное расстояние
    lower_bound = quartile_1 - (iqr * left) # считаем нижнюю границу уса. Надо понимать, что 1.5 квантильного расстояние - это стандарт. Но можно проявлять творчетво
    display(lower_bound)
    upper_bound = quartile_3 + (iqr * right) # верхняя грань уса
    display(upper_bound)
    outliers = data[(x<lower_bound) | (x > upper_bound)] #отфильтрованные данные вне усов = выбросы
    cleaned = data[(x>=lower_bound) & (x <= upper_bound)] #данные внутри усов
    return outliers, cleaned

outliers, cleaned = outliers_iqr_mod(df, 'balance')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

df = cleaned



-2241.0

4063.0

Число выбросов по методу Тьюки: 1057
Результирующее число записей: 10105


## Часть 2:  Разведывательный анализ

### Задание 1

In [1197]:
# изучите соотношение классов в ваших данных на предмет несбалансированности, проиллюстрируйте результат
# ваш код

### Задания 2 и 3

In [1198]:
#рассчитайте описательные статистики для количественных переменных, проинтерпретируйте результат
#ваш код

### Задания 4 и 5

In [1199]:
#рассчитайте описательные статистики для категориальных переменных, проинтерпретируйте результат
#ваш код
#постройте визуализации, иллюстрирующие результаты

### Задание 6

In [1200]:
# Узнайте, для какого статуса предыдущей маркетинговой кампании успех в текущей превалирует над количеством неудач.
# ваш код

### Задание 7

In [1201]:
# узнайте, в каком месяце чаще всего отказывались от предложения открыть депозит
# ваш код

### Задание 8

In [1202]:
# создайте возрастные группы и определите, в каких группах более склонны открывать депозит, чем отказываться от предложения

### Задания 9 и 10

In [1203]:
# постройте визуализации для открывших и неоткрывших депозит в зависимости от семейного статуса

In [1204]:
# постройте визуализации для открывших и неоткрывших депозит в зависимости от образования

In [1205]:
# постройте визуализации для открывших и неоткрывших депозит в зависимости от вида профессиональной занятости

### Задание 11

In [1206]:
# постройте сводную таблицу, чтобы определить люди с каким образованием и семейным статусом наиболее многочисленны
#(если рассматривать тех, кто открыл депозит)

## Часть 3: преобразование данных

### Задание 1

In [1207]:
# преобразуйте уровни образования

### Задания 2 и 3

In [1208]:
# преобразуйте бинарные переменные в представление из нулей и единиц

### Задание 4

In [1209]:
# создайте дамми-переменные

### Задания 5 и 6

In [1210]:
# постройте корреляционную матрицу и оцените данные на предмет наличия мультиколлинеарности

### Задания 7 и 8

In [1211]:
#X = df.drop(['deposit'], axis=1)
#y = df['deposit']
 
#X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 42, test_size = 0.33)

In [1212]:
# рассчитайте необходимые показатели

### Задание 9

In [1213]:
# с помощью SelectKBest отберите 15 наиболее подходящих признаков

### Задание 10

In [1214]:
# нормализуйте данные с помощью minmaxsxaler

# Часть 4: Решение задачи классификации: логистическая регрессия и решающие деревья

### Задание 1

In [1215]:
# обучите логистическую регрессию и рассчитайте метрики качества

### Задания 2,3,4

In [1216]:
# обучите решающие деревья, настройте максимальную глубину

### Задание 5

In [1217]:
# подберите оптимальные параметры с помощью gridsearch

# Часть 5: Решение задачи классификации: ансамбли моделей и построение прогноза

### Задание 1

In [1218]:
# обучите на ваших данных случайный лес

### Задания 2 и 3

In [1219]:
# используйте для классификации градиентный бустинг и сравните качество со случайным лесом

### Задание 4

In [1220]:
# объедините уже известные вам алгоритмы с помощью стекинга 

### Задание 5

In [1221]:
# оцените, какие признаки демонстрируют наибольшую  важность в модели градиентного бустинга

### Задания 6,7,8

In [1222]:
# реализуйте оптимизацию гиперпараметров с помощью Optuna